In [7]:
!mkdir wordPieceTokenizer

# Library import

In [1]:
from tqdm import tqdm, tqdm_notebook
tqdm.pandas()

import transformers
import pandas as pd
import nltk
import re
import pickle
import string

In [2]:
import sys
sys.version

'3.8.0 (default, Nov  6 2019, 16:00:02) [MSC v.1916 64 bit (AMD64)]'

In [3]:
import torch
torch.cuda.is_available()

True

In [4]:
from typing import Dict, List, Optional
from filelock import FileLock
import os
import json
import pickle
import random
import time
import warnings
import torch


from transformers.utils import logging
from transformers import LineByLineTextDataset
from transformers import BertConfig, BertForPreTraining, BertTokenizerFast, DataCollatorForLanguageModeling
from transformers.tokenization_utils import PreTrainedTokenizer
from tokenizers import BertWordPieceTokenizer #bertformaskedLM

from torch.utils.data.dataset import Dataset

logger = logging.get_logger(__name__)


In [5]:
transformers.__version__

'4.20.0'

# Data Load

In [9]:
pd.__version__

'1.4.2'

In [7]:
data=pd.read_pickle('./Data/fur_train_data.pkl')
data

,pre_review_
0,할인 받아서 좋은 가격에 구매했어요 남편이 이것만 사용해서 자주구매 합니다 좋아요
1,보통
2,아직 사용전이긴 하나 말 그대로 요긴하게 쓰일 듯하네요 어묵꽂이 할 때 필요해서 주...
3,만족합니다 전 리터 시켰고 요 솔찍히 빨래가 저 같은 경우 많을 때 가 잇 옅 적을...
4,좋아요 탈착할 수 있어 더욱 좋아요
...,...
1042767,일 구입 후 차례 기분 좋게 사용 했습니다 사용 후 조리대위에서 뚜껑 분리하여 세...
1042768,수고하세요
1042769,저희 집은 원목식탁이라 아이들 식탁 매트로 사용하려고 구매했어요 사이즈가 측정 후 ...
1042770,강블리 물맛이 좋네요


In [8]:
doc = data['pre_review_'].to_list()

In [9]:
# train
with open('./Data/fur/final_fur_train.txt', 'w', encoding='utf-8') as f:
    for idx, line in tqdm_notebook(enumerate(doc[:-20000])):
#         for line in lines.split('. '):
#             line = re.sub('[%s]' % re.escape(string.punctuation), '', line)
#             line = re.sub(' +', ' ', line)
        f.write(str(line) + '\n')
    if idx != len(doc[:-20000])-1:
        f.write('\n')

C:\Users\delab_7\AppData\Local\Temp\ipykernel_9368\1359118793.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, line in tqdm_notebook(enumerate(doc[:-20000])):


0it [00:00, ?it/s]

In [10]:
# validation
with open('./Data/fur/final_fur_validation.txt', 'w', encoding='utf-8') as f:
    for idx, line in tqdm_notebook(enumerate(doc[-20000:-10000])):
#         for line in lines.split('. '):
#             line = re.sub('[%s]' % re.escape(string.punctuation), '', line)
#             line = re.sub(' +', ' ', line)
        f.write(str(line) + '\n')
    if idx != len(doc[-20000:10000])-1:
        f.write('\n')

C:\Users\delab_7\AppData\Local\Temp\ipykernel_9368\3341214965.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, line in tqdm_notebook(enumerate(doc[-20000:-10000])):


0it [00:00, ?it/s]

In [11]:
# test
with open('./Data/fur/final_fur_test.txt', 'w', encoding='utf-8') as f:
    for idx, line in tqdm_notebook(enumerate(doc[-10000:])):
#         for line in lines.split('. '):
#             line = re.sub('[%s]' % re.escape(string.punctuation), '', line)
#             line = re.sub(' +', ' ', line)
        f.write(str(line) + '\n')
    if idx != len(doc[-10000:])-1:
        f.write('\n')

C:\Users\delab_7\AppData\Local\Temp\ipykernel_9368\3402576644.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, line in tqdm_notebook(enumerate(doc[-10000:])):


0it [00:00, ?it/s]

# Modeling

In [6]:
from transformers import AutoTokenizer, AutoModelWithLMHead, BertForMaskedLM
tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")

In [7]:
model = BertForMaskedLM.from_pretrained("beomi/kcbert-base")

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
# model = BertForPreTraining(config=config)
model.num_parameters()

108950064

In [9]:
data_collator = DataCollatorForLanguageModeling(    # [MASK] 를 씌우는 것은 저희가 구현하지 않아도 됩니다! :-)
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [10]:
print("Token ids 0 = {}".format(tokenizer.convert_ids_to_tokens(0)))
print("Token ids 1 = {}".format(tokenizer.convert_ids_to_tokens(1)))
print("Token ids 2 = {}".format(tokenizer.convert_ids_to_tokens(2)))
print("Token ids 3 = {}".format(tokenizer.convert_ids_to_tokens(3)))
print("Token ids 4 = {}".format(tokenizer.convert_ids_to_tokens(4)))

Token ids 0 = [PAD]
Token ids 1 = [UNK]
Token ids 2 = [CLS]
Token ids 3 = [SEP]
Token ids 4 = [MASK]


In [16]:
import time
start = time.time()

dataset1 = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path='./Data/fur/final_fur_train.txt',
    block_size=300,
)

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

D:\Anaconda3\envs\review\lib\site-packages\transformers\data\datasets\language_modeling.py:121: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Creating features from dataset file at ./Data/fur/final_fur_train.txt


time : 76.78861427307129


In [11]:
dataset2 = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path='./Data/fur/final_fur_validation.txt',
    block_size=300,
)

In [12]:
dataset3 = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path='./Data/fur/final_fur_test.txt',
    block_size=300,
)

In [18]:
import time
start = time.time()  # 시작 시간 저장

torch.save(dataset1, './Data/fur/RevBERT_fur_train.pkl')
torch.save(dataset2, './Data/fur/RevBERT_fur_validation.pkl')
torch.save(dataset3, './Data/fur/RevBERT_fur_test.pkl')

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

time : 104.95450854301453


In [11]:
import time
start = time.time()  # 시작 시간 저장

dataset1 = torch.load('./Data/fur/RevBERT_fur_train.pkl')
dataset2 = torch.load('./Data/fur/RevBERT_fur_validation.pkl')
dataset3 = torch.load('./Data/fur/RevBERT_fur_test.pkl')

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간


time : 77.47024011611938


In [12]:
for example in dataset3.examples[0:1]:
    print(example)

{'input_ids': tensor([    2,  8294,  1983,  4168,  4046, 21964,  8110,  2346,  2381, 15067,
        15681,  9007,  2458,   595,  4113, 26349,  9260,     3])}


# Train

In [13]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='RevBERT_fur_model_output(300)',
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=50,
    per_device_eval_batch_size=50,
    load_best_model_at_end=True,
#     save_steps=1000, # step 수마다 모델을 저장
#     save_total_limit=2, # 마지막 두 모델 빼고 과거 모델은 삭제
#     logging_steps=100
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator, # 밥을 어떻게 떠먹여줄지
    train_dataset=dataset1, # 밥이 뭔지
    eval_dataset=dataset2 
)

In [14]:
# wiki 전체 데이터로 학습 시, 1 epoch에 9시간 정도 소요됩니다!! 

In [15]:
torch.cuda.empty_cache()

In [16]:
trainer.train() 

D:\Anaconda3\envs\review\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1022772
  Num Epochs = 10
  Instantaneous batch size per device = 50
  Total train batch size (w. parallel, distributed & accumulation) = 100
  Gradient Accumulation steps = 1
  Total optimization steps = 102280
D:\Anaconda3\envs\review\lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
D:\Anaconda3\envs\review\lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not 

Epoch,Training Loss,Validation Loss
1,2.306200,2.196736
2,2.131300,2.028296
3,2.042500,1.958388
4,1.975300,1.904566
5,1.905300,1.869639
6,1.871300,1.846041
7,1.821600,1.790462
8,1.773800,1.749942
9,1.745000,1.715889
10,1.734500,1.735123


***** Running Evaluation *****
  Num examples = 10000
  Batch size = 100
Saving model checkpoint to RevBERT_fur_model_output(300)\checkpoint-10228
Configuration saved in RevBERT_fur_model_output(300)\checkpoint-10228\config.json
Model weights saved in RevBERT_fur_model_output(300)\checkpoint-10228\pytorch_model.bin
D:\Anaconda3\envs\review\lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
D:\Anaconda3\envs\review\lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')
***** Running Evaluation *****
  Num examples = 10000
  Batch size = 100
Saving model checkpoint to RevBERT_fur_model_output(300)\checkpoint-20456
Configuration saved in RevBERT_fur_model_output(300)\checkpoint-20456\config.

TrainOutput(global_step=102280, training_loss=1.9682160232418628, metrics={'train_runtime': 96660.3699, 'train_samples_per_second': 105.811, 'train_steps_per_second': 1.058, 'total_flos': 1.009044764922096e+18, 'train_loss': 1.9682160232418628, 'epoch': 10.0})